# Sad TP

## Library's

In [1]:
import re
import findspark
findspark.init()
from pyspark import *
from pyspark.sql import *
from graphframes import *
import operator

## Preprocessing

The preprocessig part is done using only python 

In [2]:
txt_file="data.txt"

In [3]:
file = open(txt_file)
clean_file=""


In [4]:
    for line in file:
        #removing empty lines and spacial caracters
        if line!="\n":
            clean_file+=re.sub('[^A-Za-z0-9\s]+', '',line)

In [5]:
clean_file.replace("\n"," ")
f = open('preprocessed-data.txt', 'w')

In [6]:
f.write(clean_file)

6246720

# Processing

### Reading file

In [17]:
words=spark.sparkContext.textFile("preprocessed-data.txt").flatMap(lambda line: line.split(" "))
words.take(5)

['The', 'Project', 'Gutenberg', 'EBook', 'of']

### Building the table of paires

we will give for each word an id then wee will join them by their keys (the key is incremented by 2)

In [5]:
y=words.zipWithIndex().map(lambda x: ( x[1]*2,x[0]))
y.take(5)

[(0, 'The'), (2, 'Project'), (4, 'Gutenberg'), (6, 'EBook'), (8, 'of')]

In [6]:
x=y.map(lambda x: (x[0]+2, x[1]))
x.take(5)

[(2, 'The'), (4, 'Project'), (6, 'Gutenberg'), (8, 'EBook'), (10, 'of')]

In [11]:
xy=x.join(y).map(lambda x: x[1])
xy.take(3)

[('The', 'Project'), ('Project', 'Gutenberg'), ('Gutenberg', 'EBook')]

Counting the pairs frequency #(Xi,Yj) 

In [29]:
psfq=xy.map(lambda x: (x,1)).reduceByKey(operator.add)
psfq.take(3)

[(('when', 'viewing'), 2), (('the', 'whole'), 451), (('for', 'the'), 1274)]

Counting the pairs frequency of words #(Yj)

In [30]:
wsfq=words.map(lambda word: (word, 1)).reduceByKey(operator.add)
wsfq.take(3)

[('The', 7106), ('Project', 221), ('Gutenberg', 98)]

Counting the pairs relative frequency of words #(Xi,Yj) / #(Yj)

-To do this task we add the word Yj to the RDD who has the pairs frequency so we can join him with the RDD who contains the words frequency

In [32]:
psrfq=wsfq.join(psfq.map(lambda x:(x[0][1],x,x[1]))).map(lambda x:(x[1][1][0],x[1][1][1]/x[1][0]))
psrfq.take(3)

[(('About', 'Project'), 0.00904977375565611),
 (('phrase', 'Project'), 0.03619909502262444),
 (('official', 'Project'), 0.00904977375565611)]

### 1a. The 1000 most frequent pairs (Xi , Yj) sorted by:(Yj)

The number of elements to show

In [44]:
n=3

In [45]:
psfq.sortBy(lambda x: (x[1],x[0][1]), False).toDF().show(n)

+---------+-----+
|       _1|   _2|
+---------+-----+
|     [, ]|24066|
|[of, the]|12439|
|[in, the]| 5754|
+---------+-----+
only showing top 3 rows



### 1b. The 1000 most frequent pairs (Xi , Yj) sorted by:#(Xi,Yj) / #(Yj)

In [48]:
psrfq.sortBy(lambda x: (x[1],x[0][1]), False).toDF().show(100)

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|    [the, zygomatic]|1.0|
|   [process, zygoma]|1.0|
|   [tender, zonenot]|1.0|
|          [dat, zis]|1.0|
|          [of, zinc]|1.0|
|       [boss, zides]|1.0|
|            [, zeus]|1.0|
|     [my, zeropoint]|1.0|
|        [and, zeres]|1.0|
|       [the, zenith]|1.0|
|          [and, zen]|1.0|
|       [to, zakuska]|1.0|
|    [with, youvodka]|1.0|
|       [to, youturn]|1.0|
|      [to, youthose]|1.0|
|[his, youthfulnes...|1.0|
|[shrewd, youthfully]|1.0|
|      [told, youthe]|1.0|
|[frightened, yousit]|1.0|
|       [in, yourshe]|1.0|
|[othersthe, young...|1.0|
|[splendid, youngs...|1.0|
|         [to, youis]|1.0|
|      [beg, yougive]|1.0|
|      [for, youeven]|1.0|
|    [tell, youPeter]|1.0|
|      [for, youJems]|1.0|
|       [, yorkshire]|1.0|
|            [, york]|1.0|
|            [, yore]|1.0|
|             [, yon]|1.0|
|       [were, yoked]|1.0|
|[quite, yieldedwent]|1.0|
|       [beaux, yeux]|1.0|
|

### 2. The 1000 most frequent pairs of each word (Yj)

In [55]:
psrfq.sortBy(lambda x: (x[1],x[0][1]), False).distinct().take(5)

[(('tender', 'zonenot'), 1.0),
 (('', 'zeus'), 1.0),
 (('my', 'zeropoint'), 1.0),
 (('and', 'zeres'), 1.0),
 (('and', 'zen'), 1.0)]

## Experiments

To be continued...